In [1]:
import tensorflow as tf
import os, warnings
import pandas as pd
warnings.filterwarnings('ignore')
import numpy as np
from logp.zq.io.smi2feat import load_feat_from_csv
from logp.zq.io.load_dataframe import load_pd
from logp.zq.model.model import potential_model
from sklearn.metrics import mean_absolute_error,mean_squared_error
from logp.zq.io.base import sparse_batch

In [2]:
#filename = 'logp/DATASETS/sf_ave_PhysProp_MD_weights_charge.csv'
#dataset = pd.read_csv(filename)
#dataset.shape

In [3]:
#results = load_pd(filename)

In [4]:
# load the predicting data in martel
filename_martel = 'logp/DATASETS/Martel/MD_max/weight_q/sf_max_Martel_MD_weights_charge.csv'
dataset_martel = pd.read_csv(filename_martel)
dataset_martel.shape
results_martel = load_pd(filename_martel)
dataset_martel = lambda: load_feat_from_csv(results_martel,24,split={'train':10,'test':0},shuffle=False)
martel_train = lambda: dataset_martel()['train'].apply(sparse_batch(5))

In [5]:
# load the predicting data in star
filename_star = 'logp/DATASETS/star_nonstar/MD_max/weight_q/sf_max_star_MD_weights_charge.csv'
dataset_star = pd.read_csv(filename_star)
dataset_star.shape
results_star = load_pd(filename_star)

dataset_star = lambda: load_feat_from_csv(results_star,24,split={'train':10,'test':0},shuffle=False)
star_train = lambda: dataset_star()['train'].apply(sparse_batch(5))

In [6]:
# load the predicting data in huusk
filename_nonstar = 'logp/DATASETS/Huuskonen/MD_max/weight_q/sf_max_Huuskonen_MD_weights_charge.csv'
dataset_nonstar = pd.read_csv(filename_nonstar)
dataset_nonstar.shape
results_nonstar = load_pd(filename_nonstar)

dataset_nonstar = lambda: load_feat_from_csv(results_nonstar,24,split={'train':10,'test':0},shuffle=False)
nonstar_train = lambda: dataset_nonstar()['train'].apply(sparse_batch(5))

In [7]:
lrs = [1e-2,1e-3, 1e-4, 1e-5]

logps_martel = {}
logps_star = {}
logps_nonstar = {}

for lr in lrs:

    logps_martel['lr_{}'.format(lr)] = []
    logps_star['lr_{}'.format(lr)] = []
    logps_nonstar['lr_{}'.format(lr)] = []

    for x in range(10):
        # define the parameters
        params = {
        'model_dir':'MD_max_weight_q_PhysProp/75_75_75/lr_{}/kfold_{}'.format(lr,x),
        'network':'bpnn',
        'network_params':{
        'nn_spec':{1:[75,75,75],6:[75,75,75],7:[75,75,75],8:[75,75,75]}},
        'model_params': {'learning_rate': lr}
        }
        #split the dataset into train and test
        ftrain = 'logp/DATASETS/PhysProps/MD_max/weight_q/sf_max_PhysProp_MD_weights_q_train_{}.csv'.format(x)
        ftest  = 'logp/DATASETS/PhysProps/MD_max/weight_q/sf_max_PhysProp_MD_weights_q_test_{}.csv'.format(x)
        dtrain = load_pd(ftrain)
        dtest = load_pd(ftest)
        dataset_train = lambda: load_feat_from_csv(dtrain,24,split={'train':10,'test':0})
        dataset_test = lambda: load_feat_from_csv(dtest,24,split={'train':0,'test':10})
        # yeild the train and test `input_fn`
        train = lambda: dataset_train()['train'].repeat().shuffle(1000).apply(sparse_batch(100))
        test = lambda: dataset_test()['test'].repeat().apply(sparse_batch(50))
    
        # initialize the model
        model = potential_model(params)
        # define the hooks
        early_stopping = tf.estimator.experimental.stop_if_no_decrease_hook(model, metric_name='loss',
                                                              max_steps_without_decrease=1000,
                                                              min_steps=100)
        # train and evaluate the model
        train_spec = tf.estimator.TrainSpec(input_fn=train, hooks=[early_stopping] ,max_steps=200000)
        eval_spec = tf.estimator.EvalSpec(input_fn=test, steps=400)
        tf.estimator.train_and_evaluate(model, train_spec, eval_spec)
    
        # predict the data in martel 
        preds_martel = model.predict(input_fn=martel_train)
        preds_star = model.predict(input_fn=star_train)
        preds_nonstar = model.predict(input_fn=nonstar_train)
    
        logp_martel = []
        for pred in preds_martel:
            logp_martel.append(pred['logp'])
        logps_martel['lr_{}'.format(lr)].append(logp_martel)
    
        logp_star = []
        for pred in preds_star:
            logp_star.append(pred['logp'])
        logps_star['lr_{}'.format(lr)].append(logp_star)
    
        logp_nonstar = []
        for pred in preds_nonstar:
            logp_nonstar.append(pred['logp'])
        logps_nonstar['lr_{}'.format(lr)].append(logp_nonstar)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'MD_max_weight_q_PhysProp/75_75_75/lr_0.01/kfold_0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator

INFO:tensorflow:global_step/sec: 102.033
INFO:tensorflow:loss = 27.58749, step = 5000 (0.980 sec)
INFO:tensorflow:global_step/sec: 102.79
INFO:tensorflow:loss = 19.545504, step = 5100 (0.974 sec)
INFO:tensorflow:global_step/sec: 101.281
INFO:tensorflow:loss = 1.5702643, step = 5200 (0.987 sec)
INFO:tensorflow:global_step/sec: 101.659
INFO:tensorflow:loss = 15.221638, step = 5300 (0.984 sec)
INFO:tensorflow:global_step/sec: 101.933
INFO:tensorflow:loss = 7.5365725, step = 5400 (0.982 sec)
INFO:tensorflow:global_step/sec: 102.071
INFO:tensorflow:loss = 11.135885, step = 5500 (0.979 sec)
INFO:tensorflow:global_step/sec: 101.044
INFO:tensorflow:loss = 6.84471, step = 5600 (0.990 sec)
INFO:tensorflow:global_step/sec: 101.373
INFO:tensorflow:loss = 9.5188875, step = 5700 (0.987 sec)
INFO:tensorflow:global_step/sec: 101.846
INFO:tensorflow:loss = 7.9826813, step = 5800 (0.982 sec)
INFO:tensorflow:global_step/sec: 101.564
INFO:tensorflow:loss = 11.975878, step = 5900 (0.984 sec)
INFO:tensorflo

KeyboardInterrupt: 

In [ ]:
logps_martel_ave = {}
logps_star_ave = {}
logps_nonstar_ave = {}

for lr in lrs:
    # average the results
    logps_martel_tmp = np.array(logps_martel['lr_{}'.format(lr)])
    logps_martel_ave['lr_{}'.format(lr)] = np.mean(logps_martel_tmp,axis=0)
    #logps_martel_ave

    logps_star_tmp = np.array(logps_star['lr_{}'.format(lr)])
    logps_star_ave['lr_{}'.format(lr)] = np.mean(logps_star_tmp,axis=0)

    logps_nonstar_tmp = np.array(logps_nonstar['lr_{}'.format(lr)])
    logps_nonstar_ave['lr_{}'.format(lr)] = np.mean(logps_nonstar_tmp, axis=0)

In [ ]:
# get the true value of martel
logp_exp_martel  = []
for x in results_martel:
    logp_exp_martel.append(x[3])

# get the true value of star
logp_exp_star = []
for x in results_star:
    logp_exp_star.append(x[3])
    
# get the true value of nonstar
logp_exp_nonstar = []
for x in results_nonstar:
    logp_exp_nonstar.append(x[3])

In [ ]:
# calculate the mae, mse, and rmse of martel
for lr in lrs:
    mae_martel = mean_absolute_error(logps_martel_ave['lr_{}'.format(lr)],logp_exp_martel)
    mse_martel = mean_squared_error(logps_martel_ave['lr_{}'.format(lr)],logp_exp_martel)
    print('#-----{}------#'.format(lr))
    print(mae_martel,mse_martel,np.sqrt(mse_martel))

In [ ]:
# calculate the mae, mse, and rmse of star
for lr in lrs:
    mae_star = mean_absolute_error(logps_star_ave['lr_{}'.format(lr)],logp_exp_star)
    mse_star = mean_squared_error(logps_star_ave['lr_{}'.format(lr)],logp_exp_star)
    print('#-----{}------#'.format(lr))
    print(mae_star,mse_star,np.sqrt(mse_star))

In [ ]:
# calculate the mae, mse, and rmse of nonstar
for lr in lrs:
    mae_nonstar = mean_absolute_error(logps_nonstar_ave['lr_{}'.format(lr)],logp_exp_nonstar)
    mse_nonstar = mean_squared_error(logps_nonstar_ave['lr_{}'.format(lr)],logp_exp_nonstar)
    print('#-----{}------#'.format(lr))
    print(mae_nonstar,mse_nonstar,np.sqrt(mse_nonstar))